In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # important line if cannot load api key

## Getting the APIFY api key from the .env file
os.environ["APIFY_API_TOKEN"] = os.getenv('APIFY_API_TOKEN')

# print to check!
# print(os.environ["APIFY_API_TOKEN"])

In [ ]:
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

# Initialize the ApifyWrapper
apify = ApifyWrapper()

def create_booking_loader():
    return apify.call_actor(
        actor_id="voyager/booking-scraper",
        run_input={
            "search": "Brampton, ON",
            "checkIn": "2024-08-23",
            "checkOut": "2024-08-25",
            "adults": 8,  # 4 couples
            "children": 0,
            "rooms": 4,  # 4 rooms for 4 couples
            "currency": "CAD",
            "language": "en-us",
            "maxPages": 1
        },
        dataset_mapping_function=lambda item: Document(
            page_content=f"Hotel: {item['name']}\nPrice: {item['price']}\nRating: {item['rating']}\nAddress: {item['address']}\nDescription: {item['description']}",
            metadata={"url": item["url"], "source": "Booking.com"}
        )
    )

# Use the loader
loader = create_booking_loader()

# Load the data
data = loader.load()

print(data)